인증시스템
- 회원가입, 로그인같은 사용자 정보를 활용하는 기능을 통틀어 인증시스템이라고 부름

1. customuser
    - django는 기본적으로 로그인을 처리할 수 있는 기본 user모델 지원
    - 기본 user 모델은 id 비밀번호, 이름과 같은 최소한의 정보만을 지원
    - 사용자 모델에 추가 정보를 저장하고 싶다면 별도의 user 모델 구성해야함
    
2. users 앱 설치
    2-1. settings.py에  user 앱 추가

3. AbstractUser : Django가 CustomUser 모델을 만들기 위해 제공하는 기본 유저 형태를 가진 모델 클래스
    3-1. AbstractUser 상속받으면 자동으로 다음 필드들이 모델에 추가됨
        - username : 사용자명, 로그인할 때 아이디
        - password : 비밀번호
        - first_name : 이름
        - last_name : 성
        - email 
        - is_staff : 관리자 여부
        - is_active : 활성화 여부
        - date_joined : 가입일시
        - last_login : 마지막 로그인 일시
    - 커스텀 유저 모델을 사용하는 경우, 어떤 모델을 user 모델로 사용하는 지 settings.py에 정의해야함
```
    # 사용법 : {App 이름}.{Model 이름}
    AUTH_USER_MODER = "users.User"
```

4. admin.py 등록
```
from django.contrib.auth.admin import UserAdmin
from users.models import User


@admin.register(User)
class CustomerUsersAdmin(UserAdmin):
    fieldsets = [
        (None, {"fields":("username", "password")}),
        ("개인정보",{"fields":("last_name","first_name","email")}),
        ("추가필드",{"fields":("profile_image","short_description")}),
        (
            "권한",
            {
                "fields":(
                    "is_active",
                    "is_staff",
                    "is_superuser"
                )
            }
        ),
        ("중요한 일정", {"fields":("last_login","date_joined")}),
    ]


```

customuser에 필드 추가
- customuser에 프로필 이미지와 소개글 필드 추가


- templates - 회원 / 공통 / 피드
- static
- media

1. settings.py에 추가
```
STATIC_URL = "static/"
STATICFILES_DIRS = [BASE_DIR / "static"]

MEDIA_URL = "media/"
MEDIA_ROOT = BASE_DIR / "media"
```

2. urls.py 추가
```
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    path("admin/", admin.site.urls),
]

urlpatterns += static(
    prefix = settings.MEDIA_URL,
    document_root = settings.MEDIA_ROOT,
)
```

2-1. templates 폴더 pystagram 밖에 생성
settings.py  수정

```TEMPLATES_DIR = BASE_DIR / "templates"
TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [TEMPLATES_DIR],
        "APP_DIRS": True,
```

3. view.py 추가
```

from django.shortcuts import render

def index(request):
    return render(request, "index.html")


```


### 로그인 / 피드 페이지 기본 구조

- pystagram 접속하면, 로그인 중이라면 바로 피드페이지
- 로그인 되지 않았거나 처음 접속한 경우 로그인페이지

- 두가지 조건에 맞도록  view에서 동작 제어해야함
    - 이미 사용자가 브라우저 로그인했다면
    - > 피드 (새 글 목록)페이지 보여줌

    - 로그인 한적 없다면 (또는 로그아웃 했다면)
    - > 로그인 페이지를 보여줌

- page not found(404)에러
    - 페이지 찾을 수 없음
    - 요청한 url에 대한 페이지를 찾을 수 없다는 응답코드
    - url 해석할 때 매칭되는 패턴을 찾지 못했을때 발생

#### 현재 정의된 URL들은
1. "admin/"으로 시작하는 URL -> 관리자 페이지
2. 공백 문자열 -> index view
3. "media/"로 시작하는 URL -> 사용자가 업로드한 정적파일
 - 새로 추가한 users/urls.py 에 정의한 URL은 여기에 나타나지 않음
     - 새 urls.py는 RootURLcomf에 등록해야 함

##### pystagram/urls.py 에서 사용된 include 함수는  users/로 시작하는 URL을 users/urls.py에서 처리하게 하며, 127.0.0.1:8000/users/login/URL은 다음의 과정을 거쳐 view에 전달됨

1. pystagram/urls.py
    - /users/로 시작하는 URL을 users/urls.py로 전달
    
2. users/urls.py
    - 나머지 login/ 부분을 login_view로 전달
    
3. urls에서 전달해준 요청을 view가 처리


## 피드 페이지(Feeds)

1. 글 목록 보여줄 posts앱 생성
2. view는 feeds 함수
3. url : 127.0.0.1:8000/posts/feeds/
    3-1. posts 접두어로 posts 앱과 연결
4. Template : templates/posts/feeds.html

#### 로그인 여부에 따른 접속 제한

- 로그인 여부에 따라 동작을 구분하려면 요청을 보낸 사용자의 정보가 필요
- View함수에 전달된 요청(request)에서 사용자 정보는 request.user 속성으로 가져올 수 있으며, 가져온 request.user가 로그인된 사용자인지 여부는 is_authenticated 속성으로 확인할 수 있음

- posts.views.py
```
def feeds(request):
    # 요청으로부터 사용자 정보 가져옴
    user = request.user

    #가져온 사용자가 "로그인 했는지"여부 가져옴
    is_authenticated = user.is_authenticated

    print("user:", user)
    print("is_authenticated:", is_authenticated)
    
```

```
def feeds(request):
    # 요청에 포함된 사용자가 로그인 하지 않은 경우
    if not request.user.is_authenticated:
        # /users/login/ URL로 이동시킴
        return redirect("/users/login")
    
```

- users.views.py
```
# 이미 로그인되어 있다면 
    if request.user.is_authenticated:
        return redirect("/posts/feeds/")
    
```

LoginForm 인스턴스 생성에 딕셔너리를 전달하면 Form 클래스는 정의된 필드들에 올바른 값이 들어왔는지, 제약 조건을 지킨 데이터가 들어왔는지 검사

    - is_vaild 메서드를 호출할 때 이 검사가 실행되며, 검사 결과가 올바른지를 True/False로 return
    - 전달된 데이터를 검증하는 것 외에도, Form 클래스는 Template에서 input 요소를 생성하는 기능도 수행

- Form은 Template에 input요소들을 생성할 때와 자신에게 전달된 데이터를 검증할때 사용
    - 일반적으로 data없이 생성된 Form은 Template에 form 정보를 전달하기 위해 사용
    - data 인수를 채운채로 생성된  Form은 해당 data 유효성 검증하기 위해 사용

- Form.is_valid
    - is_valid 메서드를 실행하기 전에는 form의 cleaned_data에 접근할 수 없음
    - Form 클래스를 사용해 데티너를 받았다면 반드시 is_valid 호출해야함

- authenticate 
    - 주어진 값에 해당하는 사용자가 있는지 판단
    - username, password에 해당하는 사용자가 있다면 함수 실행결과로 User인스턴스 반환되며, 없다면 안됨
    
    - authenticate 함수 실행결과가 user객체라면 입력한 값(credentials; 자격증명)에 해당하는 사용자가 리턴

- login
    - 브라우저에 해당 사용자를 유지시켜주는 기능
    - authenticate가 단순히 입력한 username / password에 해당하는 사용자가 있는지 검사하고 User 객체를 돌려준다면, login 함수는 우리가 웹사이트에 로그인했다면 기대하는 로그인 상태로 변환 및 유지 기능 담당
    
        - login 함수 호출에는 현재 요청(request)객체와 사용자(User)객체가 필요

## 로그아웃 구현 및 로그인 개선

- 로그아웃은 로그인과 달리 입력값을 받지 않으므로 Template없이 View만으로 구현
    - 로그아웃 기본 구조
        - View : logout_view
        - Url : 127.0.0.1:8000/users/logout
        - Template : 없음

1. users.view.py

```
def logout_view(request):
    # logout 함수 호출에 request 전달
    logout(request)

    #logout 처리 후, 로그인 페이지로 이동
    return redirect("/users/login")
```

2. users.urls.py
```
    path("logout/", views.logout_view),
```

3. feeds.html
```
<h1> Feeds </h1>
        <a href="/users/logout/">로그아웃</a>
```

### 회원가입 
- 회원가입은 Post 객체를 만드는 글쓰기와 유사하게 입력받은 정보로 User 객체를 만드는 작업
    - 다만, user객체는 다른 일반적인 model클래스와 다른 특징을 갖고있어, 몇가지 더 고려할 사항들이 있음

    - 회원가입 기본구조
        - view : signup
        - URL : /users/signup/
        - Template : templates/users/signup.html

- users.forms.py
```
class Signup(forms.Form):
    username = forms.CharField()
    password1 = forms.CharField(widget=forms.PasswordInput)
    password2 = forms.CharField(widget=forms.PasswordInput)
    profile_image = forms.ImageField()
    short_description = forms.CharField()
```

- Django 는 User 비밀번호 변형해서 저장

    - 사용자가 입력한 비밀번호 암호화하지 않고 DB 저장은 보안 문제발생, 대한민국 개인정보 보호법 위반
    - 그래서 Django의 user모델에는 비밀번호를 변형해서 저장하는 기능이 내장되어있음
         - 사용자 정보는 create_user() 메소드 사용해야함

#### user 생성하기
1. 고려사항
    - 비밀번호와 비밀번호 확인의 값이 같아야함
    - 같은 id 사용하는 user는 생성 불가 및 오류전달
    

#### SignupForm 내부에서 데이터 유효성 검사
- Form 클래스는 기본적으로 탑재된 유효성 검사 외에 추가적인 검사를 하도록 커스텀 가능
    - 회원가입시 입력받는 데이터 username, password1, password2 이 데이터에 대한 데이터 검증 필요

- 하나의 필드에 대한 유효성 검사는 clean_{필드명} 메서드가 담당
- Form에 전달된 전체 data에 대한 유효성 검사는 clean메서드가 담당
    - 예) 하나의 필드인 username은 clean_username메서드에 검증 로직을 작성
    - 예) 비밀번호는 두개의 필드 내용을 동시에 사용해야하므로 (password1, passord2) 하나의 필드 데이터만 가지고 검증할 수 없음. 이떄는 전체 데이터를 사용할 수 있는 clean메서드 사용

- users.views.py

```
def signup(request):
    if request.method == "POST":
        form = SignupForm(data=request.POST, files=request.FILES)
        if form.is_valid():
            username = form.cleaned_data["username"]
            password1 = form.cleaned_data["password1"]
            password2 = form.cleaned_data["password2"]
            profile_image = form.cleaned_data["profile_image"]
            short_description = form.cleaned_data["short_description"]
            print(username)
            print(password1, password2)
            print(profile_image)
            print(short_description)

        context = {"form":form}
        return render(request, "users/signup.html", context)
    
    # SignupForm 인스턴스 생성, Template 전달
    form = SignupForm()
    context = {"form":form}
    return render(request, "users/signup.html", context)
```

- users.view.py 수정(고려사항)
```
def signup(request):
    if request.method == "POST":
        form = SignupForm(data=request.POST, files=request.FILES)
        if form.is_valid():
            username = form.cleaned_data["username"]
            password1 = form.cleaned_data["password1"]
            password2 = form.cleaned_data["password2"]
            profile_image = form.cleaned_data["profile_image"]
            short_description = form.cleaned_data["short_description"]

            # 2개 비밀번호 같은지 검사
            if password1 != password2:
                form.add_error("password2",
                               "비밀번호와 비밀번호 확인란의 값이 다릅니다")
                
            # 해당 id 중복인지 검사
            if User.objects.filter(username=username).exists():
                form.add_error("username", 
                               "이미 사용중인 id 입니다")
                
            #에러 존재한다면 에러 포함한 form 사용해 회원가입 페이지 다시 렌더링
            if form.errors:
                context = {"form":form}
                return render(request, "users/signup.html", context)
            
            #에러 없다면 사용자 생성 후 로그인 후 피드페이지 이동
            else:
                user = User.objects.create_user(
                    username= username,
                    password = password1,
                    profile_image = profile_image,
                    short_description = short_description,

                )
                login(request, user)
                return redirect("/posts/feeds")

    
        context = {"form":form}
        return render(request, "users/signup.html", context)
    
    # GET  요청시에는 빈 Form 보여줌
    else:
        # SignupForm 인스턴스 생성, Template 전달
        form = SignupForm()
        context = {"form":form}
        return render(request, "users/signup.html", context)
```

- foms.py
- import 추가
```
from users.models import User
from django.core.exceptions import ValidationError
```
```
def clean_username(self):
        username = self.cleaned_data["username"]

        if User.objects.filter(username=username).exists():
            raise ValidationError(f"입력한 사용자명({username})은 이미 사용 중입니다")
        
        return username
```
```
def clean(self):
        password1 = self.cleaned_data["password1"]
        password2 = self.cleaned_data["password2"]

        if password1 != password2:
            # password2 필드에 오류추가
            self.add_error("password2",
                           "비밀번호와 비밀번호 확인값이 다릅니다")
```


- clean_username 은 SignupForm에 전달된 username 키에 해당하는 값을 검증할 때 사용됨
    - 검증하려는 필드 데이터에 접근할 때는 ```"self.cleaned_data["필드명"]```에서 값을 가져옴
    - 이 값을 사용할 수 있다면 함수에서 return해주고, 유효하지 않다면 ValidationError를 발생시킴
    - clean_username에서 ValidationError 발생시키는 것은 Form.add_error("username", {입력한 에러메세지}) 호출한 것과 같다
    
- 두개 이상의 필드값을 동시 비교해야 할 대는 전체 데이터의 검증을 수행하는 clean 메서드 내부에 로직 구현
- clean_{필드명} 메서드와는 달리, clean메서드는 마지막에 값을 리턴하지 않아도 됨

- users.veiws.py 수정

```
def signup(request):
    if request.method == "POST":
        form = SignupForm(data=request.POST, files=request.FILES)
        #Form 에러가 없다면 곧바로 User 생성하고 로그인 후 피드페이지  이동
        if form.is_valid():
            username = form.cleaned_data["username"]
            password1 = form.cleaned_data["password1"]
            profile_image = form.cleaned_data["profile_image"]
            short_description = form.cleaned_data["short_description"]

           
            user = User.objects.create_user(
                username= username,
                password = password1,
                profile_image = profile_image,
                short_description = short_description,

            )
            login(request, user)
            return redirect("/posts/feeds")

        #form에 에러가 있다면, 에러를 포함한 Form 사용해 회원가입 페이지 보여줌
        else:
            context = {"form": form}
            return render(request,"users/signup.html", context)
       
```

#### 깔끔한 코딩으로 바꾸기

- forms.py
```
  def save(self):
        username = self.cleaned_data["username"]
        password1 = self.cleaned_data["password1"]
        profile_image = self.cleaned_data["profile_image"]
        short_description = self.cleaned_data["short_description"]

        user = User.objects.creat_user(
            username = username,
            password = password1,
            profile_image = profile_image,
            short_description = short_description,

        )
    return user
```

- views.py
```
    if request.method == "POST":
        form = SignupForm(data=request.POST, files=request.FILES)
        #Form 에러가 없다면 form의 save()메서드로 사용자 생성
        if form.is_valid():
            user = form.save()
            login(request, user)
            return redirect("/posts/feeds")
            
         # GET  요청시에는 빈 Form 보여줌
    else:
        # SignupForm 인스턴스 생성, Template 전달
        form = SignupForm()
        
    context = {"form":form}
    return render(request, "users/signup.html", context)
```

### 템플릿 모듈만들기 

- Template 확장하는 {% extends %}태그
    - {% extends "템플릿 경로" %} 태그는 입력한 경로의 template을 기반으로 새 Template 생성
    
- 공통되는 부분은 남겨두고, 템플릿마다 변경되는 부분은 {% block content %}{% endblock %} 태그로 치환
    - {% block %}영역은 이 템플릿을 확장하는 하위 템플릿에서 변경 가능한 부분
    - base.html에는 하나의 block 밖에 없으므로, base.html을 확장(extends)하는 하위 템플릿들은 content block 내의 영역만 편집 가능.
    - 나머지 부분은 base.html의 변경사항 따라가게 됨
    
- content block 내부 채우려면
{% block content %}로 블록 영역이 시작함을 알리고
{% endblock %}으로 영역이 끝났음을 선언해 주어야한다.

## 피드 구성하기

- model : 게시글, 댓글, 이미지
- users와 연결

#### posts.models.py
```
class Post(models.Model):
    user = models.ForeignKey("users.user", 
                             verbose_name = "작성자", 
                             on_delete=models.CASCADE)
    content = models.TextField("내용", blank=True)
    created = models.DateTimeField("작성일시", auto_now_add=True)
```


```
class PostImage(models.Model):
    post = models.ForeignKey(Post,
                             verbose_name="포스트",
                             on_delete=models.CASCADE)
    photo = models.ImageField("사진", upload_to="post")
```



```
class Comment(models.Model):
    user = models.ForeignKey("users.User",
                             verbose_name="작성자",
                             on_delete=models.CASCADE)
    post = models.ForeignKey(Post,
                             verbose_name="포스트",
                             on_delete=models.CASCADE)
    content = models.TextField("내용")
    created = models.DateTimeField("작성일시", auto_now_add=True)
```

- admin.py 등록
```
from posts.models import Post, PostImage, Comment

# Register your models here.
class CommentInline(admin.TabularInline):  ## 포스트볼때 댓글 같이볼 수 있음
    model = Comment
    extra = 1   

@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    list_display=[
        "id",
        "content",
        "created",
    ]
    
    inlines = [
    CommentInline,

@admin.register(PostImage)
class PostImageAdmin(admin.ModelAdmin):
    list_display = [
        "id",
        "post",
        "photo",
    ]

@admin.register(Comment)
class CommentAdmin(admin.ModelAdmin):
    list_display = [
        "id",
        "post",
        "content",
        "created",
    ]
```

- 포스트 사진도 보이게하기

```
from django.contrib.admin.widgets import AdminFileWidget
from django.db import models
from django.utils.safestring import mark_safe

# Register your models here.
class CommentInline(admin.TabularInline):
    model = Comment
    extra = 1

# AdminFileWidget : 관리자페이지에서 '파일선택'버튼 보여주는 부분
class InlinesImageWidget(AdminFileWidget):
    def render(self, name, value, attrs=None, renderer=None):
        html = super().render(name, value, attrs, renderer)
        if value and getattr(value, "url", None):
            html = mark_safe(f"<img src='{value.url}' width = '150' height='150'>"
                             ) + html
        return html

class PostImageInline(admin.TabularInline):
    model = PostImage
    extra = 1
    formfield_overrides = {
        models.ImageField:{
            "widget" : InlinesImageWidget,
        }
    }

```

- 간단하게 변경하기
    - pip install django-admin-thumbnails
    
    
```
import admin_thumbnails


@admin_thumbnails.thumbmail("photo")
class PostImageInline(admin.TabularInline):
    model = PostImage
    extra = 1
```

## 이미지 슬라이드

- java css 추가후
- base.html
-> ``` <link rel="stylesheet" href="{% static 'splide/splide.css' %}">
        <script src="{% static 'splide/splide.js' %}"></script>```

- feeds.html
```
<!--이미지 슬라이드 영역-->
            <div class="post-images splide">
                <div class="splide__track">
                    <ul class="splide__list">
                        {% for image in post.postimage_set.all %}
                            {% if image.photo %}
                            <li class="splide__slide">
                                <img src="{{ image.photo.url }}">
                            </li>
                            {% endif %}
                        {% endfor %}
                    </ul>
                </div>
            </div>

```
```
<script>
    const elms = document.getElementsByClassName("splide")
    for (let i =0; i < elms.length; i++) {
        new Splide(elms[i]).mount();
    }
</script>

```

### 댓글 작성
- 블로그프로젝트에서는 목록화면에서는 댓글을 보여주기만하며, 댓글 작성은 글 내부에서만 가능했지만

- 피드 페이지에서는 댓글달기 기능을 한 화면에서 각 post마다 구현

- 사용자의 입력을 받는 input 직접 만들수도 있지만 Form 클래스를 사용하는 것이 Django기본 규칙

- 이전까지는 forms.Form 클래스를 사용했지만 ModelForm 클래스를 사용하면 DB테이블에 해당하는 모델 클래스와 연관된 기능들을 제공함


- Django  모델에 있는 필드인 models.CharField()나 models.InterField()는 ModelForm에서 forms.CharField()나 forms.IntegerField()와 같은 Form에서 사용하는 Field로 자동 변환됨

- ModelForm 인스턴스에서 save() 호출하면 전달받은 데이터를 사용해서 지정된 모델 인스턴스를 생성
    - 오류가 발생
    - 발생한 오류는 posts_comment 테이블의 post_id는 NULL 허용하지않는다는 메세지
    
- 오류 해결방법 두가지
    - 1. CommentForm으로 Comment 객체를 일단 만들되, 메모리상에 객체를 만들고 필요한 데이터를 나중에 채우기
    - 2. CommentForm에 Null허용하지 않는 모든 필드를 선언하고, 인스턴스 생성시 유효한 데이터를 전달
    

- ModelForm에 모든 필드를 지정하면 별도의 작업 없이 save()만 호출하면 새 모델 인스턴스가 생기므로 fields리스트에 모든 필드를 지정하는게 맞다고 생각할 수 있음

하지만 Form에서 전달받는 데이터는 **사용자가 입력한 데이터**임

- comment를 생성하기 위해 필요한 데이터는 3가지
    - 1. 어떤 글의 댓글인지(post)
    - 2. 어떤 사용자의 댓글인지(user)
    - 3. 어떤 내용을 가지고 있는지(content)
        - 여기서 사용자가 입력하는 데이터는 1번, 3번
        - 어떤 사용자가 댓글을 생성했는지 사용자가 입력한 데이터에 있으면 안되는 값이며, 시스템에서 자동으로 입력되야 함
        
        -- 그러므로 CommentForm은 post와 content만을 전달받은 값으로 지정해야되며, 작성자 정보인 user는 시스템에서 채워야함

### 댓글 작성 처리를 위한 view 구현

- 지금까지는 form을 사용한 POST 요청으로 받은 데이터를 같은 페이지에서 처리했음
- 많은 역할을 하나의 view에서 처리하게 되면 코드를 유지보수하기 어려워짐
- 댓글 작성 form에서 전송한 데이터는 별도의 댓글 작성 view에서 처리

- View에 require_POST 데코레이터를 사용하면 오로지 POST 유형의 요청만 처리하며, 이외의 유형의 요청에는 405 Method Not Allowed 응답 돌려줌

- 지금까지 form 태그에서 지정해본 속성값
    - method : GET, POST 중 어떤 방식으로 데이터 전달할지
    - enctype : 기본값과 파일 전송을 위한 값 중 선택
    
- 이번에 사용할 속성값은 action
    - action : 이 form의 요청을 어디로 보낼지를 지정
        - 비어있는 경우에는 현재 브라우저의 url 사용
        

### 작성 완료 후 원하는 Post위치로 이동

- 글에 댓글을 추가한 후 피드 페이지의 최상단이 아니라 댓글을 추가한 글로 돌아올 수 있도록 구현
    - HTML 요소의 id 속성 활용
    
- URL 뒤 #{HTML 요소의 id} 입력하면, 그 id 가진 요소의 위치로 이동하게 됨

#### 글의 댓글 수 표시
- Post에 몇개의 Comment가 연결되었는지 표시
    - 연결된 객체의 정보가 전부 필요하지 않고 개수만 필요하다면 QuerySet의 count 메서드 사용하면 좋음
    

#### 댓글 삭제

- 삭제할 댓글의 id 정보를 받고, 받은 id에 해당하는 Comment 객체를 delete()메서드로 삭제
    - 댓글 삭제는 db의 내용을 수정하는 명령이므로 method가 POST일때만 동작해야함
    
- Comment 작성한 소유자가 아니어도 적당한 Comment의 id값을 사용해 comment_delete에 삭제 요청을 하면 댓글을 삭제할 수 있음
    - view 함수에서 삭제요청이 들어온 Comment의 작성자가 요청한 사용자와 일치하는지 먼저 확인해야함

#### 글작성 기본구조 구현
- View : post_add
- URL : /posts/post_add
- Template : template/posts/post_add.html
    

#### PostForm 구현

- Form 이름 : PostForm
- Post 모델 사용
- content 입력받을 수 있게
- 데이터 전달방식 = POST

> post_add.html 에서 PostForm에 사용자에게 데이터를 입력받을 수 있도록 개발

### POST만들기 위해서는 내용 뿐 아니라 이미지 파일들도 필요
- 이미지 파일을 저장하는 필드는 Post모델이 아닌 PostImage 모델이 있음

- ModelForm은 기본적으로 class Meta 속성에 정의한 하나의 모델만을 생성할 수 있음
    - 이미지 파일 여러장을 추가로 받아 처리하는 기능은 가지고 있지 않음
    - 여러장의 이미지를 업로드해서 PostImage 객체를 여러개 생성하는 기능을 PostForm 과는 별도로 구성

- post_add.html
```
        <!--file 전송할 것이므로 enctype 수정-->
        <form method = "POST" enctype="multipart/form-data">
            {% csrf_token %}
            <div>
                <lable for="id_images">이미지</lable>
                <input id="id_images" name="images" type="file" multiple>
            </div>
```

- 파일 첨부 위해 ```<input>의 type을 file```로 선언
- 여러개의 파일 첨부위해 multiple 속성 추가 선언
    - multiple 속성은 선언만 하면되고 따로 값을 지정하지는 않음
    
- 파일 첨부할 것이므로 form의 enctype을 multipart/form-data지정
    - label에 있는 for 속성은 이 label이 어떤 input에 대한 설명인지 지정하는 역할
    - 값을 가리키는 input의 id 속성값을 지정해야함

- View에서 multiple 속성을 가진 file input 데이터 받기
    - Template의 images와 content 중 content는 PostForm으로 전달하고 images로 전달된 여러개의 파일을 별도로 처리해야함
    - multiple속성으로 전달한 여러 개의 파일 데이터는 request.FILES 대신 ```request.FILES.getlist("<전달된 input의 'name' 속성>")``` 으로 가져옴
    
    ```
               for image_file in request.FILES.getlist("images"):
                # request.FILES 또는 request.FILES.getlist()로 가져온 파일은
                # Model의 ImageField 부분에 곧바로 할당
                PostImage.objects.create(
                    post=post,
                    photo=image_file,
                    )
             # 모든 PostImage와 Post 생성이 완료되면
            # 피드페이지로 이동하여 생성된 Post의 위치로 스크롤되도록함
            url = f"/posts/feeds/#post-{post.id}"
            return HttpResponseRedirect(url)
                    
    ```

### 유지보수(동적URL)

- URL 변경할때 생기는 중복작업
- 1. 프로젝트가 복잡해질수록 한 URL을 여러곳에서 사용할 것이고, 변경할 부분이 많음
    - 1-1 URLconf에 있는 URL값이 변경되었을 때 자동으로 변경된 내용을 반영할 수 있다면 관리가 쉬워짐
    - 1-2 Django 에서는 이를 위해 동적 URL을 사용할 수 있는 기능을 제공함
    
< 동적 URL 생성을 위한 추가>
> 동적 URL 생성해서 사용하기 위해서는 app 별로 분리된 하위 urls.py에 app_name이라는 속성이 필요

< template을 위한 {% url %}태그 >
> {% 'URL pattern name' %}태그는 Template에서 urls.py의 내용을 이용해 동적으로 URL 생성해줌
>> URL pattern name은 {url.py에 있는 app_name}:{path()에 지정된 name}의 구조

< {% url'posts:comment_delete' comment_id=comment.id% } > 
- URL pattern name이 posts:comment_delete로 app_name이 posts인 url.py 에서 name이 comment_delete인 URL을 동적으로 생성하겠다는 의미

- 이 경로는 ```<int:comment_id>``` 부분의 값을 URL을 통해 동적으로 입력받음
    - 그러므로 이 path()를 사용해서 URL 경로를 만드려면 동적으로 입력받는 부분인 comment_id값이 필요함

< View의 동적 URL 변경>
> template에서 {% url %} 태그 사용하듯, View에서는 reverse 함수로 동적 URL 생성

```
from django.urls import reverse

url = reverse("posts:feeds") + f"#post-{comment.post.id}"
return HttpResponseRedirect(url)

```

## 해시태그 만들기
- 다대다 관계 모델
    - 다대일(n:1)관계는 한 테이블에 한 레코드가 다른 테이블의 여러레코드와 연관된 관계
    - 다대다(n:n)관계는 한 테이블에 여러 레코드가 다른 테이블의 여러레코드와 연관관계
    - 다대다(n:n)관계는 두 테이블의 연결을 정의하는 또 하나의 테이블 필요

- Post 모델에서 다대다를 선언하거나, HashTag 모델에서 다대다를 선언하거나 어느쪽 중간에 테이블이 하나 만들어진다는 결과는 같음

    - 하지만 관계에서 어느 쪽이 좀더 다른 쪽을 포함하는지에 따라 다대다를 선언하는 모델이 달라짐
    > 둘중에서 좀더 타당하게 느껴지는 쪽을 다대다를 선언하는 모델로 정함
    - > 글(Post)에 해시태그 여러개 포함하기
    - > 해시태그(HashTag)에 글 여러 개를 포함하기
    

- 관리자페이지에서 여러 해시태그를 고를 수 있지만 체크박스 형태의 UI보다 불편

- admin에 formrield_overrides 옵션을 추가하면 선택할 항목을 checkbox로 표시할 수 있음

- admin.py
```
@admin.register(HashTag)
class HashTagAdmin(admin.ModelAdmin):
    pass

```

- models.py
```
class HashTag(models.Model):
    name = models.CharField("태그명", max_length=50)

    def __str__(self):
        return self.name
    
```

- Post 모델에 tags라는 이름의 ManyToManyField 선언
    - post.tags.all()로 연결된 전체 HashTag 객체를 불러올 수 있음
    
#### 해시태그 검색 기본구조
- view : posts/views.py -> tag 함수
- url : /posts/tags/{tag의 name}/
- Template : templates/posts/tags.html

### 해시태그 생성

- 모델에 정의된 ManyToManyField에 HashTag 추가할때는 add함수 사용

- get_or_creat() : 인수로 전달하는 값에 해당하는 객체가 이미 존재한다면 DB의 내용을 가져오고, 없으면 DB에 새로 생성

    - 결과는 2개의 아이템을 가진 튜플로 반환
    - {DB에서 가져오거나 생성된 객체}, {생성여부} = Model.objects.get_or_create()

## 글 상세 페이지

- 기본구조
    - view : posts/views.py -> post_detail
    - url : 127.0.0.1:8000/posts/<post_id>/
    - Template : templeates/posts/post_detail.html

### Template 재사용
- {%include%} 태그로 <article>태그 사용
    - extends : 확장 / 기본 구조, 양식
    - include : 공통부분 포함하게 만드는 것

- 글 작성 후 이동할 위치 지정
    - 기존의 댓글 작성 후 redirect 로직
    - 댓글 작성 완료 후 피드페이지로 이동하라는 응답을 반환
    - 댓글은 피드 페이지와 글 상세 페이지 양쪽에서 작성할 수 있으므로 각각의 경우게 따라 다르게 지정
    
- {% include %}태그의 with옵션
    - 각각의 글을 나타내는 HTML 요소는 {% include 'post.html' %}로 가져오고 있으며, 댓글을 작성하는  CommentForm은 post.html Template 내에서 사용하고 있음
    - post.html을 include로 가져오면서 댓글 작성 후 이동할 URL 값을 전달해야 함
    
- {% 태그명 as 변수명 %} : 태그로 만들어진 결과값을 Template 내에서 사용할 변수 할당

- {% url 'posts:post_detail' post.id as action_redirect_to %}는 post.id 상세페이지 URL이 action_redirect_to 변수에 할당

    - action_redirect_to 변수에 할당된 상세페이지 url은 댓글 작성이 완료된 후에 브라우저에서 이동해야 할 주소

- {%include 'Template명' with 변수명=값 %}:include로 가져올 Template에 변수명으로 값을 전달
    - {%include 'posts/post.html' with action_redirect_url=action_redirect_to%}는 post.html Template 을 렌더할 때, action_redirect_url이라는 변수를 추가적으로 사용

#### Template 중복코드 제거
- 각 화면단위 기능의 기반이 되는 레이아웃은 3가지
    - 상단 내비게이션 바가 없는 레이아웃 : base.html (login / signup)
    - 내비게이션 바가 있는 레이아웃 : base_nav.html
    - 내비게이션 바가 있으며, 이미지 슬라이더 기능이 포함된 레이아웃 : base_slider.html
    

### 좋아요 기능
- 좋아요 기능은 해시태그와 같은 다대다 관계를 사용
- 해시태그는 글 생성 시 입력한 문자열을 쉼표단위로 구분해서 생성했지만
좋아요는 form과 button 으로 구성해 언제든지 추가/삭제할 수 있는 토글방식으로 구현


- 좋아요 모델, 관리자 페이지 구성
    - 좋아요 기능은 해시태그와 같은 M2M DB 구조 사용
    - 사용자가 좋아요 누른 Post와 Post에 좋아요를 누른 사용자들의 관계는 사용자의 좋아요 액션으로 만들어짐
        - 사용자쪽이 주도적이므로 User 모델에 like_posts로 ManyToManyField를 정의하고 좋아요 기능 구현

- 필드에 정의한 related_name 속성은 역방향으로 Model을 참조할때 사용하는 이름
    - User입장에서는 좋아요 한 목록을 user.like_posts.all()을 불러올 수 있으며
    - 반대로 post입장에서는 자신에게 좋아요를 누른 user목록을 post.like_users.all()로 불러올 수 있음
    
    -> 만약 related_name별도로 지정하지 않으면 {모델명의 소문자}_set으로 지정
    post.user_set이라는 이름은 어떤 조건의 user들과 연결된 것인지 알 수 없으므로 의미를 명확히 나타내기 위해 related_name 별도로 지정

좋아요 처리는 토글(toggle) 방식을 사용
- 이미 좋아요 누른 상태라면 해제
- 그렇지 않으면 좋아요상태로 만듦
- ManyToMany 연결 제거하거나 추가하는 방식으로 구현할 수 있음

- 사용자가 좋아요를 누른 Post 목록에 요청이 전달된 Post가 포함되어있는지 판단하고
    - 이미 좋재한다면 연결 삭제
    - ManyToManyField 의 remove 메서드로 연결을 삭제할 수 있음
    
- 반대로 전달된 Post이미 좋아요를 누른 목록에 속하지 않는다면
    - add로 새로운 연결을 생성
    
- 생성이든 삭제든 로직이 실행된 후에는 댓글 작성 때와 같이 next로 전달된 URL로 되돌아감


1. form의 action주소는 방금 생성한 post_like View 함수로 연결되게 하며, DB 데이터를 변경하므로 method는 POST 방식 사용

2. csrf_token 제외하면 form 내부에 아무런 input이 없음
- Post의 경우 토글 기능에는 특별히 전달할 데이터가 없고
- 이런 경우 내부 요소 없이 단순히 POST 요청만을 전달

3. {%if user in post.like_users.all%} 태그로 이 Post에 좋아요 누른 모든 User목록에 현재 로그인한 유저가 포함되는지 판단
- 좋아요 누른 상태라면 button 태그의 style 속성에 color:red; 값을 지정해 글자를 빨간색으로 바꾸어 사용자가 이 Post에 좋아요를 한 상태임을 표시

### 팔로우/ 팔로잉기능
- 팔로우/팔로잉 관계는 해시태그, 좋아요와 같이 ManyToManyField사용해 다대다 관계로 구성되나 이들과는 다른 점이 있음
- 해시태그와 좋아요는 한쪽에서의 연결은 반대쪽에서의 연결도 나타나는 대칭적(Symmetrical)인 관계
- 팔로우/팔로잉 관계는 한쪽에서의 연결과 반대쪽에서의 연결이 별도로 구분되는 비대칭적인 관계

#### 프로필 페이지
- 프로필 페이지 기본구조
    - 자신의 정보, 작성한글, 팔로우/팔로잉 수를 보여줄 페이지
    - View : users/view.py -> profile
    - URL : users/<int:users_id>/profile
    - Template: users/profile.html

- base_profile.html구성
    - 프로필 페이지에서는 사용자 정보와 사용자의 Post목록을 표시
    - 프로필 페이지의 사용자 정보는 재사용하고
    - Post목록 대신 팔로우/팔로잉 목록을 사용할 수 있도록
    - 상단에 사용자 정보를 공통으로 사용하는 기반 template인 base_profile.html 구성

- 팔로우/팔로잉 목록
    - 자신을 팔로우하는 사용자 목록
        - view: users/views.py -> followers
        - url: /users/<int:user_id>/followers
        - template: users/followers.html
        
    - 자신이 팔로우하는 사용자 목록
        - view: users/views.py -> following
        - url: /users/<int:user_id>/following
        - template: users/following.html
        

### 팔로우 토글 view

- post의 좋아요 기능과 같이 이미 팔로우 되어있다면 언팔로우를 
- 팔로우 되어있지 않다면 팔로우 목록에 추가하는 토글 기능을 사용
    - view : users/view.py -> follow
    - url : /users/<inst:user_id>/follow/
    - template : 없음